In [1]:
## Import der Module
# Pyhton Version 3.11.1
# Selenium Version 4.9.1
# webdriver_manager Version 3.8.6
# pandas Version  2.0.1
# datetime Version 5.1
# sqlalchemy Version 2.0.13
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import random
import pandas as pd
import datetime
from sqlalchemy import create_engine
import os
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
#----------------------------------------------------------------------------------
## Hilfsfunktionen
# Log File
def schreibe_log_file(log_eintrag):
    datei='log-Monster.txt' # Name der Log-Datei
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    # Überprüfen, ob die Log-Datei bereits existiert
    if not os.path.exists(datei):
        # Wenn die Log-Datei nicht existiert, wird sie erstellt und der Log-Eintrag wird hineingeschrieben
        with open(datei, 'w') as file:
            file.write(f"{timestamp}, {log_eintrag}" + '\n')
    else:
        # Wenn die Log-Datei bereits existiert, wird der Log-Eintrag an das Ende der Datei angehängt
        with open(datei, 'a') as file:
            file.write(f"{timestamp}, {log_eintrag}" + '\n')

## E-Mail - Benachrichtigung bei Fehlern
def schreibe_e_mail(message, Subject="fehlerhaft"):
    # SMTP-Server-Konfiguration
    host = "smtp.web.de"
    port = 587
    log_in_id = "dc-jobs"
    passwort = "DatacraftKurs0822!"

    # Erstellen der E-Mail-Nachricht
    msg = MIMEMultipart()
    msg['From'] = log_in_id + "@web.de" # Absender
    msg['To'] = "jobs@data-craft.de" # Empfänger
    msg['Subject'] = f'Monster Abfrage {Subject}' # Betreff der E-Mail

    msg.attach(MIMEText(message, 'plain')) # Hinzufügen des Nachrichtentextes zur E-Mail

    # Verbindung zum SMTP-Server herstellen und E-Mail senden
    with smtplib.SMTP(host=host, port=port) as mail:
        mail.starttls() # Starte TLS-Verschlüsselung
        mail.login(log_in_id, passwort) # Anmeldung am SMTP-Server
        mail.send_message(msg) # E-Mail senden

## random Wartezeit zum für seitenaufbau oder so
def wartezeit(zeit=3):
    """
    Diese Funktion fügt eine zufällige Wartezeit hinzu, bevor sie fortgesetzt wird.

    :param zeit: Die maximale Wartezeit in Sekunden (Standardwert ist 3).
    :return: None
    """
    time.sleep(random.randint(1, zeit))
#------------------------------------------------------------------------------------------------
## SQL-Server verbindung erstellen
# SQL-Server-Konfiguration
#host="datacraft-db.cf3hyz1fwdiw.eu-central-1.rds.amazonaws.com"
host='datacraft-jobs.postgres.database.azure.com'
database="postgres"
user="jobs_rw"
password="01QT3X48xCi4KXUckxDq"
tabelle_Rohdaten="jobs.rohdaten"

# Erstelle eine SQL-Verbindung mit der Datenbank
connection = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}/{database}')
#-----------------------------------------------------------------------------------------------
## Der Scraper
def scraper(jobtitel, suchort='Deutschland'):
    '''
    Der eigentliche Scraper für die Seite
    
    :param jobtitel: Der Titel des Jobs, nach dem gesucht werden soll.
    :param suchort: Der Ort, an dem nach Jobs gesucht werden soll. Standardwert ist 'Deutschland'.
    '''
    
    schreibe_log_file(f'suche nach {jobtitel}')
    
    # ChromeOptions erstellen
    chrome_options = webdriver.ChromeOptions()
    #Headless-Modus aktivieren
    #chrome_options.add_argument("--headless")

    # Browserfenster öffnen nach Einstellung in den Optionen
    try:
        driver = webdriver.Chrome(options=chrome_options)
        schreibe_log_file('Browser geoeffnet')
    except:
        schreibe_log_file('Browser konnte nicht geoeffnet werden')
        schreibe_e_mail(f'Browser konnte nicht geoeffnet werden: -- {datetime.datetime.now().strftime("%d.%m.%Y, %H:%M:%S")} --')

    # Webseite aufrufen
    try:
        wartezeit(3)
        driver.get("https://www.monster.de/")
        time.sleep(3)
    except:
        schreibe_log_file('Webseite wurde nicht geladen')
        schreibe_e_mail(f'Webseite wurde nicht geladen: -- {datetime.datetime.now().strftime("%d.%m.%Y, %H:%M:%S")} --')

    ## Cookies akzeptieren
    wartezeit(3)
    try:
        driver.find_element(By.XPATH, '//*[@id="onetrust-accept-btn-handler"]').click()
    except:
        pass
    wartezeit(3)

    ## Suchfelder finden, anwählen und befüllen
    # Suchfeld Jobbezeichnung -> finden und befüllen
    suchfeld_jobtitel=driver.find_element(By.XPATH,'//*[@id="horizontal-input-one-undefined"]')
    suchfeld_jobtitel.send_keys(jobtitel)
    wartezeit(3)

    # Suchfeld Ort -> finden und befüllen
    suchfeld_ort=driver.find_element(By.XPATH,'//*[@id="horizontal-input-two-undefined"]')
    suchfeld_ort.send_keys(suchort)

    # Anfrage mit Enter/Return abschicken
    wartezeit(3)
    suchfeld_ort.send_keys(Keys.RETURN)

    wartezeit(3)

    # Fenster größe verkleiner zum scrallen und damit auflisten der Stellenanzeigen
    driver.set_window_size(800 , 600)

    # Seite für weitere Stellenanzeigen nach unten Scrollen
    for i in range(10):
        driver.execute_script("window.scrollBy(0, 1000);")
        time.sleep(random.randint(3,4))

    # Fenster Maximieren
    driver.maximize_window() # evtl. im Headless-Modus nicht machbar

    link_liste_scraped=[]

    ## link_liste_scraped befüllen von erster Seite
    # alte bezeichnungen (sc-chibGv,)
    anzeigen=driver.find_elements(By.CLASS_NAME, 'sc-cbPlza')

    for anzeige in anzeigen:
        link_liste_scraped.append(anzeige.get_attribute('href'))

    # überprüfen ob link_liste_scraped leer ist
    # da evtl. die Class name geändert wurde
    if len(link_liste_scraped) == 0:
        schreibe_log_file('Keine Links auf gefunden')
        schreibe_e_mail('Keine Links auf Webseite gefunden:\n\n Hinweis: Classenname überprüfen für alle Stellenanzeigen!')
        return
    else:
        pass

    # Duplikate entfernen
    link_liste_scraped_ohne_duplikate = list(set(link_liste_scraped))

    # URLs aus Datenbank laden
    existing_urls = pd.read_sql_query(f"SELECT url FROM {tabelle_Rohdaten}", connection)
    schreibe_log_file(f'{len(existing_urls)} in der datenbak bereits vorhanden')

    # Filtere den DataFrame, um nur neue URLs zu behalten
    link_liste = [url for url in link_liste_scraped_ohne_duplikate if url not in existing_urls['url'].values]

    seiten_inhalt_html_liste = []
    seiten_inhalt_liste = []
    URL_Liste = []
    Datum_Liste = []

    try:
        schreibe_log_file('Abfrage begonnen')
        for link in link_liste:
            
            # Seite Aufrufen
            driver.get(link)
            wartezeit()
            
            # "Webseite ist nicht erreichbar" wird angezeigt
            try:
                driver.find_element(By.XPATH, '//*[@id="reload-button"]').click() # auf Button "neu laden" klicken
            except:
                pass
            
            # Listen befüllen
            seiten_inhalt_html_liste.append(driver.find_element(By.CLASS_NAME,
                                                                'main-layoutstyles__Layout-sc-1w7iv1n-0').get_attribute('innerHTML'))
            seiten_inhalt_liste.append(driver.find_element(By.CLASS_NAME, 
                                                           'main-layoutstyles__Layout-sc-1w7iv1n-0').text)
            URL_Liste.append(link)
            Datum_Liste.append(datetime.datetime.now())
            schreibe_log_file(f'Daten wurden gezogen: {link}')
            time.sleep(random.randint(2,10))
    except:
        schreibe_log_file("Abfrage abgebrochen !!!")
        schreibe_e_mail(f"Die Abfrage wurde abgebrochen. -- {datetime.datetime.now()} --")
    schreibe_log_file('Abfrage beendet')

    # Schließen des Browsers
    time.sleep(2)
    driver.quit()
    schreibe_log_file('Browser geschlossen')
    
    if len(set(map(len, [seiten_inhalt_html_liste,
                        seiten_inhalt_liste,
                        URL_Liste,
                        Datum_Liste]))) > 1:
        schreibe_log_file('Listen sind verschieden lang')
        schreibe_e_mail('Listen sind nicht gleich lang')
        return
    else:
        # DataFrame erstellen
        df = pd.DataFrame({"seite": "monster",
                        "seiten_inhalt_html": seiten_inhalt_html_liste,
                        "seiten_inhalt": seiten_inhalt_liste, 
                        "url": URL_Liste, 
                        "datum":Datum_Liste,
                        "storno": False
            
        })
        
    ## Daten in die Datenbank einfügen
    # Schreibe den bereinigten DataFrame in die Datenbank
    df.to_sql(name=tabelle_Rohdaten.split('.')[1],
            schema=tabelle_Rohdaten.split('.')[0],
            con=connection, if_exists='append', index=False, )
    schreibe_log_file(f'Es wurden {len(df)} Daten von Monster hinzugefügt')
    
    # Verbindung zur SQL-Datenbank schließen
    connection.dispose()
print("fertig")

fertig


In [2]:
scraper("Data Analyst")

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=115.0.5790.171)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x0082A813+48355]
	(No symbol) [0x007BC4B1]
	(No symbol) [0x006C5358]
	(No symbol) [0x006AD293]
	(No symbol) [0x0070E37B]
	(No symbol) [0x0071C473]
	(No symbol) [0x0070A536]
	(No symbol) [0x006E82DC]
	(No symbol) [0x006E93DD]
	GetHandleVerifier [0x00A8AABD+2539405]
	GetHandleVerifier [0x00ACA78F+2800735]
	GetHandleVerifier [0x00AC456C+2775612]
	GetHandleVerifier [0x008B51E0+616112]
	(No symbol) [0x007C5F8C]
	(No symbol) [0x007C2328]
	(No symbol) [0x007C240B]
	(No symbol) [0x007B4FF7]
	BaseThreadInitThunk [0x77997D59+25]
	RtlInitializeExceptionChain [0x77C7B79B+107]
	RtlClearBits [0x77C7B71F+191]


In [3]:
# Ich arbeite mit zwei Computern, und leider hat die Datei für die Suchbegriffe (Jobtitel) jeweils ein anderen Speicherort
try:
    suchbegriffe = pd.read_excel(r"C:\Users\Admin\Documents\DreamJobs\Benötigte Dateien\Jobtitel.xlsx",
                                sheet_name='Monster')
except:
    suchbegriffe = pd.read_excel(r"C:\Users\Admin\Documents\DreamJobs\Benötigte Dateien\Jobtitel.xlsx",
                                sheet_name='Monster')  
display(suchbegriffe)

,Jobtitel
0,Datenanalyst
1,Customer Data Analyst
2,Business Analyst Reporting & Data Analytics
3,Data Analyst
4,Business Analyst
5,Data Engineer and Analyst - BI Cloud
6,Machine Learning Specialist


In [4]:
print(f'{len(suchbegriffe)} Jobtitel gefunden')
for i, jobtitel in enumerate(suchbegriffe.Jobtitel):
    print(f'{i}: Suche nach "{jobtitel}", Start: {datetime.datetime.now().strftime("%H:%M:%S")} Uhr')
    scraper(jobtitel)
print("")
print("fertig")

7 Jobtitel gefunden
0: Suche nach "Datenanalyst", Start: 14:34:40 Uhr


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=115.0.5790.171)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x0082A813+48355]
	(No symbol) [0x007BC4B1]
	(No symbol) [0x006C5358]
	(No symbol) [0x006AD293]
	(No symbol) [0x0070E37B]
	(No symbol) [0x0071C473]
	(No symbol) [0x0070A536]
	(No symbol) [0x006E82DC]
	(No symbol) [0x006E93DD]
	GetHandleVerifier [0x00A8AABD+2539405]
	GetHandleVerifier [0x00ACA78F+2800735]
	GetHandleVerifier [0x00AC456C+2775612]
	GetHandleVerifier [0x008B51E0+616112]
	(No symbol) [0x007C5F8C]
	(No symbol) [0x007C2328]
	(No symbol) [0x007C240B]
	(No symbol) [0x007B4FF7]
	BaseThreadInitThunk [0x77997D59+25]
	RtlInitializeExceptionChain [0x77C7B79B+107]
	RtlClearBits [0x77C7B71F+191]
